In [7]:
import os
import torch
import numpy as np
import time
import random
from sklearn.model_selection import KFold
import argparse
import timeit
from dataset import prepare_data
import psutil
import torch
from models.model_rbgm import GNN_1
from memory_capacity_utils import gen_lag_data, get_mem_cap_from_model

In [8]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('running on GPU')
else:
    device = torch.device("cpu")
    print('running on CPU')


running on GPU


# Load the dataset

In [9]:
def get_args():
    parser = argparse.ArgumentParser(description='Args for graph predition')
    parser.add_argument('-num_folds', type=int, default=5, help='cv number')
    parser.add_argument('--num_timepoints', type=int, default=3,
                        help='Number of timepoints')
    parser.add_argument('-num_epochs', type=int, default=20, help='number of epochs')
    parser.add_argument('--lr', type=float, default=0.001, help="Learninng rate")
    parser.add_argument('--memcap_coef', type=float, default=0, help="Memory Capacity Loss Coefficient")
    parser.add_argument('-max_lag', type=int, default=35, help='Lag tao for memory capacity signals')
    parser.add_argument('-save_path',type=str,default = '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_rbgm/',help='Path to the saved results')
    args, _ = parser.parse_known_args()
    return args

In [10]:
args = get_args()

In [11]:
#dataset = np.load('datasets/oasis_data.npy')
dataset = np.load("datasets/multivariate_simulation_data_2.npy")
dataset = torch.from_numpy(dataset).squeeze()
dataset = dataset.type(torch.FloatTensor)
#dataset = np.delete(dataset,88,axis=0)

# Set up the network architecture and train-validate

**Train-validate-functions**

In [12]:
def create_directory_if_not_exists(directory):
    """
    Checks if a specified directory exists, and creates it if it doesn't.

    Args:
    - directory (str): Path of the directory to check and potentially create.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory '{directory}' was created.")
    else:
        print(f"Directory '{directory}' already exists.")

# Create the results folders
create_directory_if_not_exists(args.save_path+'real_and_predicted_graphs')
create_directory_if_not_exists(args.save_path+'train_losses/mae_losses')
create_directory_if_not_exists(args.save_path+'train_losses/reservoir_losses')
create_directory_if_not_exists(args.save_path+'train_losses/bio_losses')
create_directory_if_not_exists(args.save_path+'train_losses/tp_losses')
create_directory_if_not_exists(args.save_path+'train_losses/total_losses')
create_directory_if_not_exists(args.save_path+'test_mae_losses')
create_directory_if_not_exists(args.save_path+'test_tp_losses')
create_directory_if_not_exists(args.save_path+'test_memcap_losses')
create_directory_if_not_exists(args.save_path+'test_predicted')
create_directory_if_not_exists(args.save_path+'test_original')
create_directory_if_not_exists(args.save_path+'trained_models')

Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_rbgm/real_and_predicted_graphs' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_rbgm/train_losses/mae_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_rbgm/train_losses/reservoir_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_rbgm/train_losses/bio_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_rbgm/train_losses/tp_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_rbgm/train_losses/total_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_rbgm/test_mae_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_rbgm/test_tp_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_rbgm/test_memcap_losses' already exists.
Directory '/vol/bitbuck

In [13]:
manual_seed = 777
np.random.seed(manual_seed)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

# Reservoir signals
X_train_res_np, y_train_res_np = gen_lag_data(1000, 41, args.max_lag)
X_test_res_np, y_test_res_np = gen_lag_data(500, 42, args.max_lag)
X_train_res = torch.from_numpy(X_train_res_np).unsqueeze(1).to(device, dtype=torch.float64)
X_test_res = torch.from_numpy(X_test_res_np).unsqueeze(1).to(device, dtype=torch.float64)
y_train_res = torch.from_numpy(y_train_res_np).to(device, dtype=torch.float64)
y_test_res = torch.from_numpy(y_test_res_np).to(device, dtype=torch.float64)

In [14]:
def validation(model, validation_subjects, mem_cap_data, X_train, y_train, X_test, y_test):
    mael = torch.nn.L1Loss().to(device)
    tp = torch.nn.MSELoss().to(device)
    
    val_mae_loss = np.zeros(args.num_timepoints - 1)
    val_tp_loss = np.zeros(args.num_timepoints - 1)
    mem_cap = np.zeros(args.num_timepoints - 1)
    predicted = np.zeros((validation_subjects.shape[0], args.num_timepoints - 1, 35, 35))
    actual = np.zeros((validation_subjects.shape[0], args.num_timepoints - 1, 35, 35))
    
    model.eval()

    with torch.no_grad():
        for n_subject, data in enumerate(validation_subjects):
            input = data[0]
            for t in range(args.num_timepoints - 1):
                pred = model(input)
                val_mae_loss[t] += mael(pred, data[t + 1])
                val_tp_loss[t] += tp(pred.sum(dim=-1), data[t + 1].sum(dim=-1))
                input = pred
                
                pred_mem_cap = get_mem_cap_from_model(model, pred, 
                                                      X_train, y_train, X_test, y_test)
                actual_mem_cap = torch.tensor(mem_cap_data[n_subject, t + 1]).to(device)
                mem_cap[t] += torch.abs(pred_mem_cap - actual_mem_cap)

                predicted[n_subject, t] = pred.cpu().detach().numpy()
                actual[n_subject, t] = data[t + 1].cpu().detach().numpy()
                

    avg_val_mae_loss = val_mae_loss/len(validation_subjects)
    avg_val_tp_loss = val_tp_loss/len(validation_subjects)
    avg_val_mae_mem_cap = mem_cap/len(validation_subjects)

    return avg_val_mae_loss, avg_val_tp_loss, avg_val_mae_mem_cap, predicted, actual

In [18]:
torch.manual_seed(1)
input_weights = (torch.rand((35, 1), dtype=torch.float64) * 2.0 - 1.0).to(device)

indexes = range(args.num_folds)
kfold = KFold(n_splits=args.num_folds, shuffle=True, random_state=manual_seed)
dataset = dataset.to(device)
actual_mem_caps = np.load('datasets/mem_caps_2.npy')
f = 0

for train, test in kfold.split(range(dataset.shape[0])):
    print(
            f'------------------------------------Fold [{f + 1}/{args.num_folds}]-----------------------------------------')
    
    train_data = dataset[train]
    test_data = dataset[test]
    train_mem_cap = actual_mem_caps[train]
    test_mem_cap = actual_mem_caps[test]
    
    validation_split = int(0.8 * len(train_data))
    train_subjects = train_data[:validation_split]
    train_mem_cap_subjects = train_mem_cap[:validation_split]
    validation_subjects = train_data[validation_split:]
    validation_mem_cap_subjects = train_mem_cap[:validation_split]

    model = GNN_1(device=device, input_weights=input_weights).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    mael = torch.nn.L1Loss().to(device)
    tp = torch.nn.MSELoss().to(device)
  
    # Start measuring the epochs time
    epochs_start = time.time()
    for epoch in range(args.num_epochs):
        
        print(f'Epoch [{epoch + 1}/{args.num_epochs}]')
        # Set the model in training mode
        model.train()
        
        # this is our loss for all the data
        mae_loss_overall = []
        tp_loss_overall = []
        
        # loop through the data batches
        for data_id, data in enumerate(train_subjects):

            # zero the gradients
            optimizer.zero_grad()
            mae_loss = 0
            tp_loss = 0
         
            # loop through the time dependent adj matrices in the batches
            for t in range(args.num_timepoints - 1): 
                pred = model(data[t])
                real = data[t + 1]
       
                mae_loss += mael(pred, real)

                # Topological Loss
                tp_loss += tp(pred.sum(dim=-1), real.sum(dim=-1))

    
            # Calculate the total MAE Loss for the current batch
            mae_loss = mae_loss / (args.num_timepoints - 1)

            # Calculate the total TP Loss for the current batch
            tp_loss = tp_loss / (args.num_timepoints - 1)

            # Append to the total MAE Loss
            mae_loss_overall.append(mae_loss.item())
            tp_loss_overall.append(tp_loss.item())
            
            total_loss = mae_loss
            # Update the weights of the neural network
            total_loss.backward()
            optimizer.step()

        mae_loss_overall = np.mean(np.array(mae_loss_overall))
        tp_loss_overall = np.mean(np.array(tp_loss_overall))
        print(f"[Train] MAE Loss: {mae_loss_overall}, TP Loss: {tp_loss_overall}")
    
        avg_val_mae_loss, avg_val_tp_loss, avg_val_mae_mem_cap, _, _ = validation(model, validation_subjects, validation_mem_cap_subjects,
                                                                                  X_train_res, y_train_res, X_test_res, y_test_res)
        print(f"[Validate] MAE Loss Across Timepoints: {avg_val_mae_loss}")
        print(f"[Validate] TP Loss Across Timepoints: {avg_val_tp_loss}")
        print(f"[Validate] MAE of Mem Caps Across Timepoints: {avg_val_mae_mem_cap}")

    
    epochs_end = time.time() - epochs_start
    print()
    print(f'epochs finished with time:{epochs_end}')
    print()
    process = psutil.Process(os.getpid())
    print(f"Current memory usage: {process.memory_info().rss / 1024 ** 2:.2f} MB")
    print()
    
    avg_test_mae_loss, avg_test_tp_loss, avg_test_mem_cap, predicted, original = validation(model, test_data, test_mem_cap,
                                                                                  X_train_res, y_train_res, X_test_res, y_test_res)
    print(f"[Test] MAE Loss Across Timepoints: {avg_test_mae_loss}")
    print(f"[Test] TP Loss Across Timepoints: {avg_test_tp_loss}")
    print(f"[Test] MAE of Mem Caps Across Timepoints: {avg_test_mem_cap}")
    np.save(args.save_path+f"test_mae_losses/mae_test_loss_fold_{f}", avg_test_mae_loss)
    np.save(args.save_path+f"test_tp_losses/tp_test_loss_fold_{f}", avg_test_mae_loss)
    np.save(args.save_path+f"test_memcap_losses/memcap_test_loss_fold_{f}", avg_test_mem_cap)
    np.save(args.save_path+f"test_predicted/predicted_fold_{f}", predicted)
    np.save(args.save_path+f"test_original/original_fold_{f}", original)

    torch.save(model.state_dict(),
               args.save_path +f'trained_models/model_fold_{f}')
    f += 1
    

------------------------------------Fold [1/5]-----------------------------------------
Epoch [1/20]
[Train] MAE Loss: 0.03682604183268268, TP Loss: 3.3837401987984776
[Validate] MAE Loss Across Timepoints: [0.04122445 0.0428788 ]
[Validate] TP Loss Across Timepoints: [1.05684853 1.31210148]
[Validate] MAE of Mem Caps Across Timepoints: [0.38015679 0.6119313 ]
Epoch [2/20]
[Train] MAE Loss: 0.02663071137794759, TP Loss: 0.6869233723264188
[Validate] MAE Loss Across Timepoints: [0.03861305 0.03962728]
[Validate] TP Loss Across Timepoints: [0.92280698 1.10231352]
[Validate] MAE of Mem Caps Across Timepoints: [0.41279375 0.69727783]
Epoch [3/20]
[Train] MAE Loss: 0.02573917221161537, TP Loss: 0.6513969583902508
[Validate] MAE Loss Across Timepoints: [0.03794999 0.03910672]
[Validate] TP Loss Across Timepoints: [0.89690578 1.08858001]
[Validate] MAE of Mem Caps Across Timepoints: [0.44465315 0.70993356]
Epoch [4/20]
[Train] MAE Loss: 0.02539616341528017, TP Loss: 0.6432515233755112
[Valida

[Validate] MAE Loss Across Timepoints: [0.03605402 0.03725576]
[Validate] TP Loss Across Timepoints: [0.93740278 1.02865267]
[Validate] MAE of Mem Caps Across Timepoints: [0.6166974  0.92066734]
Epoch [10/20]
[Train] MAE Loss: 0.024281291814986616, TP Loss: 0.6443390226922929
[Validate] MAE Loss Across Timepoints: [0.03596595 0.03709522]
[Validate] TP Loss Across Timepoints: [0.93705809 1.02486539]
[Validate] MAE of Mem Caps Across Timepoints: [0.63312446 0.93238182]
Epoch [11/20]
[Train] MAE Loss: 0.02420118171721697, TP Loss: 0.6436248924583197
[Validate] MAE Loss Across Timepoints: [0.03589679 0.03700668]
[Validate] TP Loss Across Timepoints: [0.93607408 1.02048945]
[Validate] MAE of Mem Caps Across Timepoints: [0.64500465 0.93886763]
Epoch [12/20]
[Train] MAE Loss: 0.024131237238179892, TP Loss: 0.6429999399697408
[Validate] MAE Loss Across Timepoints: [0.03580323 0.03691182]
[Validate] TP Loss Across Timepoints: [0.93690407 1.021222  ]
[Validate] MAE of Mem Caps Across Timepoints:

[Train] MAE Loss: 0.02348191328928806, TP Loss: 0.594493483309634
[Validate] MAE Loss Across Timepoints: [0.03585854 0.03647979]
[Validate] TP Loss Across Timepoints: [0.89372218 1.05297899]
[Validate] MAE of Mem Caps Across Timepoints: [0.53011006 0.89177741]
Epoch [19/20]
[Train] MAE Loss: 0.023452922381693497, TP Loss: 0.5940079973079264
[Validate] MAE Loss Across Timepoints: [0.03580878 0.03643662]
[Validate] TP Loss Across Timepoints: [0.8945024  1.05466413]
[Validate] MAE of Mem Caps Across Timepoints: [0.52740076 0.89018551]
Epoch [20/20]
[Train] MAE Loss: 0.02342271506495308, TP Loss: 0.5934306741692126
[Validate] MAE Loss Across Timepoints: [0.03578483 0.03638846]
[Validate] TP Loss Across Timepoints: [0.89095449 1.04351318]
[Validate] MAE of Mem Caps Across Timepoints: [0.53529287 0.88866159]

epochs finished with time:336.5433440208435

Current memory usage: 2591.21 MB

[Test] MAE Loss Across Timepoints: [0.0346663 0.0371241]
[Test] TP Loss Across Timepoints: [0.89797487 1.1

[Validate] MAE Loss Across Timepoints: [0.03564788 0.03808855]
[Validate] TP Loss Across Timepoints: [0.8493067  1.10811305]
[Validate] MAE of Mem Caps Across Timepoints: [0.54488765 0.80380509]
Epoch [6/20]
[Train] MAE Loss: 0.024228052992839366, TP Loss: 0.6091534590814263
[Validate] MAE Loss Across Timepoints: [0.03545565 0.03781583]
[Validate] TP Loss Across Timepoints: [0.84969682 1.1008476 ]
[Validate] MAE of Mem Caps Across Timepoints: [0.55822966 0.82393428]
Epoch [7/20]
[Train] MAE Loss: 0.024085988057777286, TP Loss: 0.6072659423807636
[Validate] MAE Loss Across Timepoints: [0.03529755 0.03761618]
[Validate] TP Loss Across Timepoints: [0.85007966 1.09794092]
[Validate] MAE of Mem Caps Across Timepoints: [0.56909256 0.84102008]
Epoch [8/20]
[Train] MAE Loss: 0.023964504696778022, TP Loss: 0.6056799817597494
[Validate] MAE Loss Across Timepoints: [0.03516733 0.03743337]
[Validate] TP Loss Across Timepoints: [0.85091448 1.09196174]
[Validate] MAE of Mem Caps Across Timepoints: [